In [1]:
pip install librosa soundfile numpy sklearn pyaudio

     ------------------------------------ 214.3/214.3 kB 544.2 kB/s eta 0:00:00
     ------------------------------------ 689.4/689.4 kB 220.6 kB/s eta 0:00:00
     --------------------------------------- 14.6/14.6 MB 74.5 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------ 164.0/164.0 kB 428.2 kB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 508.2 kB/s eta 0:00:00
     --------------------------------------- 56.3/56.3 kB 35.0 kB/s eta 0:00:00
     ---------------------------------------- 7.4/7.4 MB 257.5 kB/s eta 0:00:00
     ---------------------------------------- 3.1/3.1 MB 215.2 kB/s eta 0:00:00
     ------------------------------------ 377.0/377.0 kB 118.0 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 38.6/38.6 MB 415.6 kB/s eta 0:00

In [2]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [4]:

def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [5]:

emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#DataFlair - Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [7]:

def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("H:\\New folder\\Desktop\\7th Sem\\Capstone Project II\\SPEECHEMOTIONRECOGNITION\\speech-emotion-recognition-ravdess-data\\Actor_*\\*.wav"):   
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [8]:
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

C:\Users\HP\AppData\Local\Temp\ipykernel_33316\3312442463.py:15: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\HP\AppData\Local\Temp\ipykernel_33316\3312442463.py:15: FutureWarning: Pass y=[0.0000000e+00 3.0517578e-05 3.0517578e-05 ... 0.0000000e+00 0.0000000e+00
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\HP\AppData\Local\Temp\ipykernel_33316\3312442463.py:15: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00 -3.0517578e-05 ... -3.0517578e-05
 -3.0517578e-05 -3.0517578e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\HP\Ap

In [9]:
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [10]:
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [11]:
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [12]:

model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [16]:
y_pred=model.predict(x_test)


In [17]:

accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)


print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 69.27%
